Le but de ce code est de modifier comme il se doit le data set ytrain pour avoir des données par heures et dans un carré


In [2]:
import folium
import pandas as pd
from folium.plugins import TimestampedGeoJson
from shapely.geometry import Point
import numpy as np
from math import radians, sin, cos, sqrt, atan2
from branca.colormap import linear
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import box
from scipy.spatial import cKDTree

In [ ]:
data = pd.read_csv("csv/hourly_df_semaine2229.csv")
df_real = pd.DataFrame(data)
df_real['taux_dispo'] =  (df_real['Available'] + df_real['Offline']/3)/3
df_real = df_real[['timestamp','latitude','longitude','taux_occup','taux_dispo']]
df_real['timestamp'] = pd.to_datetime(df_real['timestamp']).dt.tz_convert('UTC')
df_real.head(2)

,timestamp,latitude,longitude,taux_occup,taux_dispo
0,2020-01-22 00:00:00+00:00,48.855667,2.354089,0.666667,0.333333
1,2020-01-22 00:00:00+00:00,48.864240,2.397724,0.666667,0.333333


In [ ]:

data_traffic= pd.read_csv("csv/moyennes_par_carre_et_t1h2229.csv")
df_traffic = pd.DataFrame(data_traffic)

df_traffic.rename(columns={'t_1h': 'timestamp'}, inplace=True)
df_traffic['timestamp'] = pd.to_datetime(df_traffic['timestamp'], errors='coerce')
df_traffic['timestamp'] = df_traffic['timestamp'].dt.tz_localize('UTC')
df_traffic.head(5)


,timestamp,carre_id,moyenne_k,somme_k,count_k,lati,long
0,2020-01-22 01:00:00+00:00,0_16,0.325000,0.65000,2,48.8125,2.3375
1,2020-01-22 01:00:00+00:00,0_20,0.000000,0.00000,2,48.8125,2.3575
2,2020-01-22 01:00:00+00:00,10_1,1.478148,8.86889,6,48.8625,2.2625
3,2020-01-22 01:00:00+00:00,10_10,1.640304,18.04334,11,48.8625,2.3075
4,2020-01-22 01:00:00+00:00,10_11,2.367567,49.71890,21,48.8625,2.3125


In [5]:
# Fonction haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Rayon de la Terre en kilomètres
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

In [6]:


# Définir les limites de la grille (basé sur Paris)
lat_min, lat_max = 48.81, 48.92
lon_min, lon_max = 2.255, 2.42
step = 0.005  # Taille d'un carré (grille de 0.005°)

# Créer les carrés de la grille
latitudes = np.arange(lat_min, lat_max, step)
longitudes = np.arange(lon_min, lon_max, step)

grille_data = []

In [7]:
# Charger le fichier GeoJSON des arrondissements de Paris
arrondissements_geojson = "arrondissements.geojson"  
arrondissements_gdf = gpd.read_file(arrondissements_geojson)  # Charger les arrondissements

In [8]:
# Coordonnées de Charenton-le-Pont et du Jardin d'Acclimatation
charenton_lat, charenton_lon = 48.8337, 2.4149
jardin_lat, jardin_lon = 48.8716, 2.2611



# Distance d'exclusion (en kilomètres)
exclusion_radius = 1  # 2 km autour de ces points

# Fonction pour vérifier si un carré est proche d'un des lieux d'exclusion
def is_near_exclusion(lat, lon):
    distance_to_charenton = haversine(lat, lon, charenton_lat, charenton_lon)
    distance_to_jardin = haversine(lat, lon, jardin_lat, jardin_lon)
    return distance_to_charenton < exclusion_radius or distance_to_jardin < exclusion_radius


In [9]:
def haversine_np(lat1, lon1, lat2, lon2):
    """
    Calcul vectorisé de la distance entre deux ensembles de points (lat1, lon1) et (lat2, lon2).
    :return: Tableau NumPy des distances.
    """
    R = 6371  # Rayon de la Terre en km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c


In [12]:
def creer_grille(lat_min, lat_max, lon_min, lon_max, step):
    grille = []
    latitudes = np.arange(lat_min, lat_max, step)
    longitudes = np.arange(lon_min, lon_max, step)
    
    for i in range(len(latitudes) - 1):
        for j in range(len(longitudes) - 1):
            carre = {
                "id": f"{i}_{j}",  # Identifiant unique pour chaque carré
                "lat_min": latitudes[i],
                "lat_max": latitudes[i + 1],
                "lon_min": longitudes[j],
                "lon_max": longitudes[j + 1],
                "center_lat": latitudes[i] + step / 2,  # Centre du carré
                "center_lon": longitudes[j] + step / 2  # Centre du carré
            }
            grille.append(carre)
    return grille

In [13]:


def calculate_distances_multiple_metrics_optimized_aveck(time_series, latitudes, longitudes, df_stations, arrondissements_gdf, ):
    """
    Calcule les distances et les métriques pour les stations les plus proches, les 2 plus proches, et les 3 plus proches,
    en utilisant des opérations optimisées avec NumPy et Pandas.
    La fonction calcule aussi la moyenne de k pour chaque carré défini par lat, lon.
    
    :param time_series: Liste ou tableau des timestamps.
    :param latitudes: Liste des latitudes de la grille.
    :param longitudes: Liste des longitudes de la grille.
    :param df_stations: DataFrame contenant les données des stations (avec un 'timestamp').
    :param arrondissements_gdf: GeoDataFrame des arrondissements de Paris.
    :param df_k: DataFrame contenant les valeurs de `k` (avec 't_1h', 'lat', 'lon', 'k').
    :param square_size: Taille approximative du carré autour de chaque centre (en degrés, ajustable).
    :return: DataFrame contenant les distances et métriques pour chaque carré à chaque timestamp.
    """
    # Grille de points (centres des carrés)
    grid_lat = np.array([(latitudes[i] + latitudes[i + 1]) / 2 for i in range(len(latitudes) - 1)])
    grid_lon = np.array([(longitudes[j] + longitudes[j + 1]) / 2 for j in range(len(longitudes) - 1)])
    grid_centers = np.array(np.meshgrid(grid_lat, grid_lon)).T.reshape(-1, 2)  # Format (N, 2)
    
    # Filtrer les points de la grille en fonction de Paris
    grid_points = pd.DataFrame(grid_centers, columns=['lat', 'lon'])
    grid_points['geometry'] = grid_points.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
    grid_points = grid_points[grid_points['geometry'].apply(lambda p: arrondissements_gdf.geometry.union_all().contains(p))]
    valid_grid_centers = grid_points[['lat', 'lon']].values  # Seulement les centres valides

    results = []

    for timestamp in time_series:
        # Stations disponibles pour l'instant donné
        df_stations_at_t = df_stations[df_stations['timestamp'] == timestamp].copy()
        df_stations_at_t = df_stations_at_t[df_stations_at_t['taux_dispo'] >= 1 / 3]

        if df_stations_at_t.empty:
            continue

        # Construire un arbre k-d pour rechercher efficacement les distances
        station_coords = df_stations_at_t[['latitude', 'longitude']].values
        kdtree = cKDTree(station_coords)

        # Trouver les 3 plus proches stations pour chaque point de la grille
        distances, indices = kdtree.query(valid_grid_centers, k=3)  # distances: (N, 3), indices: (N, 3)

        # Calcul des métriques et jointure avec le DataFrame contenant 'k'
        for i, (lat, lon) in enumerate(valid_grid_centers):
            closest_1 = distances[i, 0]
            closest_2 = distances[i, :2].mean()
            closest_3 = distances[i, :3].mean()

            taux_dispo_1 = df_stations_at_t.iloc[indices[i, 0]]['taux_dispo']
            taux_dispo_2 = df_stations_at_t.iloc[indices[i, :2]]['taux_dispo'].mean()
            taux_dispo_3 = df_stations_at_t.iloc[indices[i, :3]]['taux_dispo'].mean()

            # Filtrer les stations 'k' dans le carré (lat, lon)
            lat_min = lat - square_size / 2
            lat_max = lat + square_size / 2
            lon_min = lon - square_size / 2
            lon_max = lon + square_size / 2

            
            results.append({
                'timestamp': timestamp,
                'center_lat': lat,
                'center_lon': lon,
                'distance_to_closest_1': closest_1,
                'taux_dispo_closest_1': taux_dispo_1,
                'distance_to_closest_2': closest_2,
                'taux_dispo_closest_2': taux_dispo_2,
                'distance_to_closest_3': closest_3,
                'taux_dispo_closest_3': taux_dispo_3,
                'normalized_distance': closest_1 / distances[:, 0].max() if distances[:, 0].max() > 0 else 0

            })

    return pd.DataFrame(results)



In [14]:
def creer_grille(lat_min, lat_max, lon_min, lon_max, step):
    grille = []
    latitudes = np.arange(lat_min, lat_max, step)
    longitudes = np.arange(lon_min, lon_max, step)
    
    for i in range(len(latitudes) - 1):
        for j in range(len(longitudes) - 1):
            carre = {
                "id": f"{i}_{j}",  # Identifiant unique pour chaque carré
                "lat_min": latitudes[i],
                "lat_max": latitudes[i + 1],
                "lon_min": longitudes[j],
                "lon_max": longitudes[j + 1],
                "center_lat": latitudes[i] + 0.005/2,  # Centre du carré
                "center_lon": longitudes[j] + 0.005/2 # Centre du carré
            }
            grille.append(carre)
    return grille

In [27]:
from shapely.geometry import Point
from scipy.spatial import cKDTree
import numpy as np
import pandas as pd

def calculate_distances_multiple_metrics_optimized_occup(
    time_series, df_stations, arrondissements_gdf, lat_min, lat_max, lon_min, lon_max, step=0.005
):
    """
    Calcule les distances et métriques pour les stations les plus proches à chaque point de la grille,
    en utilisant des opérations optimisées et en se basant sur la fonction `creer_grille`.
    
    :param time_series: Liste ou tableau des timestamps.
    :param df_stations: DataFrame contenant les données des stations (avec un 'timestamp').
    :param arrondissements_gdf: GeoDataFrame des arrondissements de Paris.
    :param lat_min: Latitude minimale pour créer la grille.
    :param lat_max: Latitude maximale pour créer la grille.
    :param lon_min: Longitude minimale pour créer la grille.
    :param lon_max: Longitude maximale pour créer la grille.
    :param step: Pas de la grille en degrés.
    :return: DataFrame contenant les distances et métriques pour chaque carré à chaque timestamp.
    """
    # Création de la grille à partir des limites et du pas
    grille = creer_grille(lat_min, lat_max, lon_min, lon_max, step)
    
    # Conversion de la grille en DataFrame
    grille_df = pd.DataFrame(grille)
    
    # Filtrer les points de la grille en fonction de Paris
    grille_df['geometry'] = grille_df.apply(lambda row: Point(row['center_lon'], row['center_lat']), axis=1)
    grille_df = grille_df[grille_df['geometry'].apply(lambda p: arrondissements_gdf.geometry.contains(p).any())]

    # Centres valides de la grille
    valid_grid_centers = grille_df[['center_lat', 'center_lon', 'id']].values

    results = []

    for timestamp in time_series:
        # Stations disponibles pour l'instant donné
        df_stations_at_t = df_stations[df_stations['timestamp'] == timestamp].copy()
        df_stations_at_t = df_stations_at_t[df_stations_at_t['taux_occup'] >= 1 / 3]

        if df_stations_at_t.empty:
            continue

        # Construire un arbre k-d pour rechercher efficacement les distances
        station_coords = df_stations_at_t[['latitude', 'longitude']].values
        kdtree = cKDTree(station_coords)

        # Trouver les 3 plus proches stations pour chaque point de la grille
        distances, indices = kdtree.query(valid_grid_centers[:, :2], k=3)  # distances: (N, 3), indices: (N, 3)

        # Calcul des métriques
        for i, (lat, lon, carre_id) in enumerate(valid_grid_centers):
            closest_1 = distances[i, 0]
            closest_2 = distances[i, 1]
            closest_3 = distances[i, 2]

            taux_occup_1 = df_stations_at_t.iloc[indices[i, 0]]['taux_occup']
            taux_occup_2 = df_stations_at_t.iloc[indices[i, 1]]['taux_occup']
            taux_occup_3 = df_stations_at_t.iloc[indices[i, 2]]['taux_occup']

            results.append({
                'timestamp': timestamp,
                'carre_id': carre_id,
                'center_lat': lat,
                'center_lon': lon,
                'distance_to_closest_1': closest_1,
                'taux_occup_closest_1': taux_occup_1,
                'distance_to_closest_2': closest_2,
                'taux_occup_closest_2': taux_occup_2,
                'distance_to_closest_3': closest_3,
                'taux_occup_closest_3': taux_occup_3,
                'normalized_distance': closest_1 / distances[:, 0].max() if distances[:, 0].max() > 0 else 0
            })

    return pd.DataFrame(results)


In [28]:
time_series = df_real['timestamp'].unique()

# Appeler la fonction avec les bons paramètres
result_df_occup = calculate_distances_multiple_metrics_optimized_occup(
    time_series=time_series,
    df_stations=df_real,
    arrondissements_gdf=arrondissements_gdf,
    lat_min=lat_min,
    lat_max=lat_max,
    lon_min=lon_min,
    lon_max=lon_max,
    step=step
)

In [29]:
result_df_occup.head(2)

,timestamp,carre_id,center_lat,center_lon,distance_to_closest_1,taux_occup_closest_1,distance_to_closest_2,taux_occup_closest_2,distance_to_closest_3,taux_occup_closest_3,normalized_distance
0,2020-01-22 00:00:00+00:00,1_15,48.8175,2.3325,0.008150,0.666667,0.011619,0.333333,0.014112,0.333333,0.350038
1,2020-01-22 00:00:00+00:00,1_16,48.8175,2.3375,0.006918,0.666667,0.006982,0.333333,0.010746,0.333333,0.297110


In [30]:
df_traffic.head(2)

,timestamp,carre_id,moyenne_k,somme_k,count_k,lati,long
0,2020-01-22 01:00:00+00:00,0_16,0.325,0.65,2,48.8125,2.3375
1,2020-01-22 01:00:00+00:00,0_20,0.000,0.00,2,48.8125,2.3575


In [37]:

merged_df = pd.merge(result_df_occup, df_traffic, left_on=['carre_id', 'timestamp'], right_on=['carre_id', 'timestamp'], how='inner')


In [38]:
merged_df.head(2)

,timestamp,carre_id,center_lat,center_lon,distance_to_closest_1,taux_occup_closest_1,distance_to_closest_2,taux_occup_closest_2,distance_to_closest_3,taux_occup_closest_3,normalized_distance,moyenne_k,somme_k,count_k,lati,long
0,2020-01-22 01:00:00+00:00,1_15,48.8175,2.3325,0.008150,0.666667,0.011619,0.333333,0.014112,0.583333,0.350038,2.515874,17.61112,7,48.8175,2.3325
1,2020-01-22 01:00:00+00:00,1_16,48.8175,2.3375,0.006918,0.666667,0.006982,0.333333,0.010746,0.583333,0.297110,2.962500,11.85000,4,48.8175,2.3375


In [39]:
merged_df.drop(['somme_k','count_k','center_lat','center_lon'],axis =1, inplace= True)
merged_df['moyenne_k'] = merged_df['moyenne_k'].fillna(0)

In [40]:
merged_df.head(2)

,timestamp,carre_id,distance_to_closest_1,taux_occup_closest_1,distance_to_closest_2,taux_occup_closest_2,distance_to_closest_3,taux_occup_closest_3,normalized_distance,moyenne_k,lati,long
0,2020-01-22 01:00:00+00:00,1_15,0.008150,0.666667,0.011619,0.333333,0.014112,0.583333,0.350038,2.515874,48.8175,2.3325
1,2020-01-22 01:00:00+00:00,1_16,0.006918,0.666667,0.006982,0.333333,0.010746,0.583333,0.297110,2.962500,48.8175,2.3375


In [44]:
merged_df['occup_moy_1'] = merged_df['taux_occup_closest_1'] * (merged_df['distance_to_closest_1'] / l_carre)

merged_df['occup_moy_2'] = (1 / 2) * (merged_df['taux_occup_closest_1'] * (merged_df['distance_to_closest_1'] / l_carre) 
                                      + merged_df['taux_occup_closest_2'] * (merged_df['distance_to_closest_2'] / l_carre))

merged_df['occup_moy_3'] = (1 / 3) * (merged_df['taux_occup_closest_1'] * (merged_df['distance_to_closest_1'] / l_carre) 
                                      + merged_df['taux_occup_closest_2'] * (merged_df['distance_to_closest_2'] / l_carre) 
                                      + merged_df['taux_occup_closest_3'] * (merged_df['distance_to_closest_3'] / l_carre))

merged_df.head(5)


,timestamp,carre_id,distance_to_closest_1,taux_occup_closest_1,distance_to_closest_2,taux_occup_closest_2,distance_to_closest_3,taux_occup_closest_3,normalized_distance,moyenne_k,lati,long,occup_moy_1,occup_moy_2,occup_moy_3
0,2020-01-22 01:00:00+00:00,1_15,0.008150,0.666667,0.011619,0.333333,0.014112,0.583333,0.350038,2.515874,48.8175,2.3325,0.029687,0.025425,0.031942
1,2020-01-22 01:00:00+00:00,1_16,0.006918,0.666667,0.006982,0.333333,0.010746,0.583333,0.297110,2.962500,48.8175,2.3375,0.025199,0.018957,0.024055
2,2020-01-22 01:00:00+00:00,1_17,0.003534,0.333333,0.008904,0.666667,0.009046,0.583333,0.151785,0.758611,48.8175,2.3425,0.006437,0.019435,0.022567
3,2020-01-22 01:00:00+00:00,1_18,0.005122,0.333333,0.009908,0.583333,0.012677,0.666667,0.219959,0.157334,48.8175,2.3475,0.009328,0.020453,0.029028
4,2020-01-22 01:00:00+00:00,1_20,0.011732,0.333333,0.014273,0.333333,0.016355,0.333333,0.503850,1.810644,48.8175,2.3575,0.021366,0.023680,0.025715


In [45]:
merged_df.to_csv("Final.csv")

On définit un taux d'occupation moyen par carré, selon le nombre de stations voisines comptabilisés. 

Le taux d'occupation moyen $occup_{moy,p}$ d'un carré à l'instant $t$ en comptant ses $p$ stations les plus proches est 
 $$ occup_{moy,p}(t)= \frac{1}{p} \sum_{i=1}^p occup_i(t)  \frac{distance_i}{l_{carré}} $$
où $distance_i$ est la distance du centre du carré à la $i$ème station la plus proche et $l_{carré}$ est la demie-largeur d'un carré. On calcule $l_{carré}$ ci-dessous.

In [43]:
l_carre = haversine (48.8225,2.3725,48.8225,2.375)
print('La demie-largeur d\'un carré est égale à environ {} m.'.format(int(l_carre*1000)))


La demie-largeur d'un carré est égale à environ 183 m.


In [56]:
result_df_occup['occup_moy_1']=result_df_occup['taux_occup_closest_1']*(result_df_occup['distance_to_closest_1']/l_carre)	

result_df_occup['occup_moy_2']=(1/2)*(result_df_occup['taux_occup_closest_1']*(result_df_occup['distance_to_closest_1']/l_carre) 
+ result_df_occup['taux_occup_closest_2']*(result_df_occup['distance_to_closest_2']/l_carre))	

result_df_occup['occup_moy_3']=(1/3)*(result_df_occup['taux_occup_closest_1']*(result_df_occup['distance_to_closest_1']/l_carre) 
+ result_df_occup['taux_occup_closest_2']*(result_df_occup['distance_to_closest_2']/l_carre)	
+result_df_occup['taux_occup_closest_3']*(result_df_occup['distance_to_closest_3']/l_carre))

result_df_occup.head(5)

,timestamp,center_lat,center_lon,distance_to_closest_1,taux_occup_closest_1,distance_to_closest_2,taux_occup_closest_2,distance_to_closest_3,taux_occup_closest_3,occup_moy_1,occup_moy_2,occup_moy_3
0,2020-01-22 00:00:00+00:00,48.8175,2.3325,0.829681,0.666667,0.895720,0.333333,1.278762,0.333333,3.022103,2.326714,2.327455
1,2020-01-22 00:00:00+00:00,48.8175,2.3375,0.767356,0.666667,0.583030,0.333333,1.089418,0.333333,2.795084,1.928462,1.947006
2,2020-01-22 00:00:00+00:00,48.8175,2.3425,0.381517,0.333333,0.870231,0.666667,1.003205,0.333333,0.694834,1.932319,1.897239
3,2020-01-22 00:00:00+00:00,48.8175,2.3475,0.468196,0.333333,1.045944,0.333333,1.092598,0.666667,0.852698,1.378808,2.245796
4,2020-01-22 00:00:00+00:00,48.8175,2.3575,0.979265,0.166667,1.081931,0.333333,1.758763,0.333333,0.891740,1.431100,2.021778


In [58]:
#Importation des données de trafic pour la semaine 22 janvier - 29 janvier 
df_trafic = pd.read_csv('moyennes_par_carre_et_t1h2229.csv')
df_trafic.rename(columns={'t_1h': 'timestamp'}, inplace=True)
df_trafic['timestamp'] = pd.to_datetime(df_trafic['timestamp'])

if df_trafic['timestamp'].dt.tz is None:
    df_trafic['timestamp'] = df_trafic['timestamp'].dt.tz_localize('UTC')

df_trafic.head(20)


,timestamp,carre_id,moyenne_k,somme_k,count_k,lati,long
0,2020-01-22 01:00:00+00:00,0_16,0.325000,0.65000,2,48.8125,2.3375
1,2020-01-22 01:00:00+00:00,0_20,0.000000,0.00000,2,48.8125,2.3575
2,2020-01-22 01:00:00+00:00,10_1,1.478148,8.86889,6,48.8625,2.2625
3,2020-01-22 01:00:00+00:00,10_10,1.640304,18.04334,11,48.8625,2.3075
4,2020-01-22 01:00:00+00:00,10_11,2.367567,49.71890,21,48.8625,2.3125
5,2020-01-22 01:00:00+00:00,10_12,2.206557,33.09835,15,48.8625,2.3175
6,2020-01-22 01:00:00+00:00,10_13,2.294744,29.83167,13,48.8625,2.3225
7,2020-01-22 01:00:00+00:00,10_14,1.412666,7.06333,5,48.8625,2.3275
8,2020-01-22 01:00:00+00:00,10_15,2.186280,21.86280,10,48.8625,2.3325
9,2020-01-22 01:00:00+00:00,10_16,1.849908,11.09945,6,48.8625,2.3375


In [61]:
result_df_occup['timestamp'] = pd.to_datetime(result_df_occup['timestamp'], errors='coerce')
result_df_occup['timestamp']=result_df_occup['timestamp']+ pd.Timedelta(hours=1)

df_joined = pd.merge(df_trafic, result_df_occup, left_on=['lati', 'long','timestamp'], right_on=['center_lat', 'center_lon','timestamp'], how='inner')
df_final=df_joined[['timestamp','carre_id','lati','long','moyenne_k','occup_moy_1','occup_moy_2','occup_moy_3']]
df_final.head(20)

,timestamp,carre_id,lati,long,moyenne_k,occup_moy_1,occup_moy_2,occup_moy_3
0,2020-01-22 02:00:00+00:00,10_10,48.8625,2.3075,0.713635,1.899358,2.172283,2.166594
1,2020-01-22 02:00:00+00:00,10_12,48.8625,2.3175,0.958076,1.331156,1.693438,1.711232
2,2020-01-22 02:00:00+00:00,10_13,48.8625,2.3225,1.130856,1.294264,1.764377,1.969675
3,2020-01-22 02:00:00+00:00,10_14,48.8625,2.3275,0.950666,1.089232,1.904581,2.711427
4,2020-01-22 02:00:00+00:00,10_15,48.8625,2.3325,1.148279,1.259086,1.866099,1.341779
5,2020-01-22 02:00:00+00:00,10_18,48.8625,2.3475,3.568610,0.000000,0.132873,0.376790
6,2020-01-22 02:00:00+00:00,10_19,48.8625,2.3525,2.708890,0.000000,1.399873,0.933249
7,2020-01-22 02:00:00+00:00,10_20,48.8625,2.3575,0.000000,0.000000,1.456490,0.970994
8,2020-01-22 02:00:00+00:00,10_23,48.8625,2.3725,0.995742,0.000000,1.217158,0.811439
9,2020-01-22 02:00:00+00:00,10_24,48.8625,2.3775,1.368693,1.570103,0.785052,2.998575


In [62]:
df_final.to_csv('k_et_occup_carre.csv')

In [87]:
#result_df.to_csv("test2.csv")
